In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
population = pd.read_csv("fr_population.csv")
cereal = pd.read_csv("fr_cereales.csv")
sous_alimentation = pd.read_csv('fr_sousalimentation.csv')
vegetal = pd.read_csv('fr_vegetaux.csv')
animal = pd.read_csv('fr_animaux.csv')

# Question 1 : Calculez le nombre d'humains sur terre 

In [3]:
world_population = 1000 * population["Valeur"].sum()
world_population

8413993000

La population mondiale de 2013 est largement supérieure à celle de nos jours ce qui est une erreur. On constate que la population de la chine est comptée en double. Suppression de la valeur de la population globale de la chine pour garder celle des sous-régions. 

In [4]:
indexChine = population[ population['Symbole'] == 'A' ].index
population.drop(indexChine, inplace=True)
nb_people = 1000 * population["Valeur"].sum()
nb_people

6997326000

Le résultat obtenu semble plus cohérent avec la population mondiale en 2013

# Question 2:

In [5]:
# Création d'un dataframe végétaux en fonction des pays et des produits
vegetal_produit = vegetal.pivot_table(index=['Zone','Produit'], columns='Élément', values=['Valeur'])
vegetal_produit.fillna(0, inplace=True)

#Disponibilité intérieure du blé en France
ble = vegetal_produit.loc['France','Blé']
dispo_int_ble_fr = ble['Valeur']['Disponibilité intérieure']
dispo_int_ble_fr

20298.0

In [6]:
dispo_int_ble_fr_2 = ble['Valeur']['Production'] + ble['Valeur']['Importations - Quantité'] - ble['Valeur']['Exportations - Quantité'] + ble['Valeur']['Variation de stock']
dispo_int_ble_fr_2

20298.0

In [7]:
population_pivot = population.pivot_table(index='Zone', columns='Élément', values='Valeur')
population_pivot['Population totale'] = population_pivot['Population totale'] * 1000
population_fr = population_pivot.loc[('France'),'Population totale']

dispo_int_ble_fr_3 = ble['Valeur']['Semences'] + ble['Valeur']['Pertes'] + ble['Valeur']['Aliments pour animaux'] + (((ble['Valeur']['Disponibilité alimentaire en quantité (kg/personne/an)']*population_fr)/1000)/1000) + ble['Valeur']['Traitement'] +ble['Valeur']['Autres utilisations (non alimentaire)']
dispo_int_ble_fr_3

20298.07313

On constate qu'à un chiffre après la virgule dispo_int_ble_fr = dispo_int_ble_fr_1 = dispo_int_ble_fr_2

# Création d'un Dataframe général

In [8]:
# création d'une colonne type dans les dataframes 
vegetal['Type'] = 'vegetal'
animal['Type'] = 'animal'
cereal['is_cereal'] = True

vegetal = vegetal[vegetal['Zone'] != 'Chine' ].copy()
animal = animal[animal['Zone'] != 'Chine'].copy()
population = population[population['Zone'] != 'Chine'].copy()

#Union des deux Dataframes animal et vegetal
main_df = pd.concat([animal, vegetal])
# Création d'un tableau croisé dynamique
main_df = main_df.pivot_table(index=['Zone','Produit','Code Produit', 'Type','Code zone', 'Année'], columns='Élément', values=['Valeur'] )
main_df = main_df.reset_index()
anim_vegetal = main_df.copy()
# Ajout du dataframe Population
main_df = main_df.merge(population, on='Zone', how="left")
#Ajout du dataframe cereal
# remplacer les valeurs manquantes par 0
main_df.fillna(0, inplace=True)
main_df['Code produit'] = main_df['Code Produit','']


# Création d'une colonne 'Population_totale'
main_df['Population_totale'] = main_df['Valeur']*1000
main_df = main_df.drop(['Zone', 'Code zone','Code Domaine', 'Domaine' ,'Valeur' ,'Code Élément', ('Année', ''), 'Élément', 'Code Produit', 'Produit', 'Code année', 'Unité', 'Symbole', 'Description du Symbole'], axis=1)


main_df

,"(Zone, )","(Produit, )","(Code Produit, )","(Type, )","(Code zone, )","(Valeur, Aliments pour animaux)","(Valeur, Autres utilisations (non alimentaire))","(Valeur, Disponibilité alimentaire (Kcal/personne/jour))","(Valeur, Disponibilité alimentaire en quantité (kg/personne/an))","(Valeur, Disponibilité de matière grasse en quantité (g/personne/jour))",...,"(Valeur, Importations - Quantité)","(Valeur, Nourriture)","(Valeur, Pertes)","(Valeur, Production)","(Valeur, Semences)","(Valeur, Traitement)","(Valeur, Variation de stock)",Année,Code produit,Population_totale
0,Afghanistan,Abats Comestible,2736,animal,2,0.0,0.0,5.0,1.72,0.20,...,0.0,53.0,0.0,53.0,0.0,0.0,0.0,2013,2736,30552000
1,Afghanistan,"Agrumes, Autres",2614,vegetal,2,0.0,0.0,1.0,1.29,0.01,...,40.0,39.0,2.0,3.0,0.0,0.0,0.0,2013,2614,30552000
2,Afghanistan,Aliments pour enfants,2680,vegetal,2,0.0,0.0,1.0,0.06,0.01,...,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2013,2680,30552000
3,Afghanistan,Ananas,2618,vegetal,2,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2013,2618,30552000
4,Afghanistan,Bananes,2615,vegetal,2,0.0,0.0,4.0,2.70,0.02,...,82.0,82.0,0.0,0.0,0.0,0.0,0.0,2013,2615,30552000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15600,Îles Salomon,Viande de Suides,2733,animal,25,0.0,0.0,45.0,4.70,4.28,...,0.0,3.0,0.0,2.0,0.0,0.0,0.0,2013,2733,561000
15601,Îles Salomon,Viande de Volailles,2734,animal,25,0.0,0.0,11.0,3.34,0.69,...,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2013,2734,561000
15602,Îles Salomon,"Viande, Autre",2735,animal,25,0.0,0.0,0.0,0.06,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2013,2735,561000
15603,Îles Salomon,Vin,2655,vegetal,25,0.0,0.0,0.0,0.07,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2013,2655,561000


# Question 3:  Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines

In [9]:
# Disponibilité alimentaire kcal par pays 
main_df['disp_alim_kcal_pays'] = main_df['Valeur', 'Disponibilité alimentaire (Kcal/personne/jour)'] * main_df['Population_totale'] * 365

# Disponibilité alimentaire kg par pays 
main_df['disp_alim_kg_an'] = (main_df['Valeur', 'Disponibilité de protéines en quantité (g/personne/jour)'] / 1000)*365
main_df['disp_alim_kg_pays'] = main_df['disp_alim_kg_an'] * main_df['Population_totale']
main_df

,"(Zone, )","(Produit, )","(Code Produit, )","(Type, )","(Code zone, )","(Valeur, Aliments pour animaux)","(Valeur, Autres utilisations (non alimentaire))","(Valeur, Disponibilité alimentaire (Kcal/personne/jour))","(Valeur, Disponibilité alimentaire en quantité (kg/personne/an))","(Valeur, Disponibilité de matière grasse en quantité (g/personne/jour))",...,"(Valeur, Production)","(Valeur, Semences)","(Valeur, Traitement)","(Valeur, Variation de stock)",Année,Code produit,Population_totale,disp_alim_kcal_pays,disp_alim_kg_an,disp_alim_kg_pays
0,Afghanistan,Abats Comestible,2736,animal,2,0.0,0.0,5.0,1.72,0.20,...,53.0,0.0,0.0,0.0,2013,2736,30552000,5.575740e+10,0.28105,8586639.60
1,Afghanistan,"Agrumes, Autres",2614,vegetal,2,0.0,0.0,1.0,1.29,0.01,...,3.0,0.0,0.0,0.0,2013,2614,30552000,1.115148e+10,0.00730,223029.60
2,Afghanistan,Aliments pour enfants,2680,vegetal,2,0.0,0.0,1.0,0.06,0.01,...,0.0,0.0,0.0,0.0,2013,2680,30552000,1.115148e+10,0.01095,334544.40
3,Afghanistan,Ananas,2618,vegetal,2,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,2013,2618,30552000,0.000000e+00,0.00000,0.00
4,Afghanistan,Bananes,2615,vegetal,2,0.0,0.0,4.0,2.70,0.02,...,0.0,0.0,0.0,0.0,2013,2615,30552000,4.460592e+10,0.01825,557574.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15600,Îles Salomon,Viande de Suides,2733,animal,25,0.0,0.0,45.0,4.70,4.28,...,2.0,0.0,0.0,0.0,2013,2733,561000,9.214425e+09,0.51465,288718.65
15601,Îles Salomon,Viande de Volailles,2734,animal,25,0.0,0.0,11.0,3.34,0.69,...,0.0,0.0,0.0,0.0,2013,2734,561000,2.252415e+09,0.41610,233432.10
15602,Îles Salomon,"Viande, Autre",2735,animal,25,0.0,0.0,0.0,0.06,0.00,...,0.0,0.0,0.0,0.0,2013,2735,561000,0.000000e+00,0.01460,8190.60
15603,Îles Salomon,Vin,2655,vegetal,25,0.0,0.0,0.0,0.07,0.00,...,0.0,0.0,0.0,0.0,2013,2655,561000,0.000000e+00,0.00000,0.00


## Question 4 :

In [10]:
# Calcul du ratio kcal/kg
main_df['ratio kcal/kg'] = (main_df['Valeur', 'Disponibilité alimentaire (Kcal/personne/jour)'] * main_df['Population_totale'] *365) /(main_df['Valeur', 'Nourriture']*1000000)

#poids total 
total_weight = main_df['Valeur', 'Disponibilité alimentaire en quantité (kg/personne/an)']* main_df['Population_totale']
# Calcul du pourcentage en protéine 
main_df['pourcentage_proteine'] = main_df['disp_alim_kg_pays']/total_weight
main_df.fillna(0, inplace=True)
main_df

,"(Zone, )","(Produit, )","(Code Produit, )","(Type, )","(Code zone, )","(Valeur, Aliments pour animaux)","(Valeur, Autres utilisations (non alimentaire))","(Valeur, Disponibilité alimentaire (Kcal/personne/jour))","(Valeur, Disponibilité alimentaire en quantité (kg/personne/an))","(Valeur, Disponibilité de matière grasse en quantité (g/personne/jour))",...,"(Valeur, Traitement)","(Valeur, Variation de stock)",Année,Code produit,Population_totale,disp_alim_kcal_pays,disp_alim_kg_an,disp_alim_kg_pays,ratio kcal/kg,pourcentage_proteine
0,Afghanistan,Abats Comestible,2736,animal,2,0.0,0.0,5.0,1.72,0.20,...,0.0,0.0,2013,2736,30552000,5.575740e+10,0.28105,8586639.60,1052.026415,0.163401
1,Afghanistan,"Agrumes, Autres",2614,vegetal,2,0.0,0.0,1.0,1.29,0.01,...,0.0,0.0,2013,2614,30552000,1.115148e+10,0.00730,223029.60,285.935385,0.005659
2,Afghanistan,Aliments pour enfants,2680,vegetal,2,0.0,0.0,1.0,0.06,0.01,...,0.0,0.0,2013,2680,30552000,1.115148e+10,0.01095,334544.40,5575.740000,0.182500
3,Afghanistan,Ananas,2618,vegetal,2,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,2013,2618,30552000,0.000000e+00,0.00000,0.00,0.000000,0.000000
4,Afghanistan,Bananes,2615,vegetal,2,0.0,0.0,4.0,2.70,0.02,...,0.0,0.0,2013,2615,30552000,4.460592e+10,0.01825,557574.00,543.974634,0.006759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15600,Îles Salomon,Viande de Suides,2733,animal,25,0.0,0.0,45.0,4.70,4.28,...,0.0,0.0,2013,2733,561000,9.214425e+09,0.51465,288718.65,3071.475000,0.109500
15601,Îles Salomon,Viande de Volailles,2734,animal,25,0.0,0.0,11.0,3.34,0.69,...,0.0,0.0,2013,2734,561000,2.252415e+09,0.41610,233432.10,1126.207500,0.124581
15602,Îles Salomon,"Viande, Autre",2735,animal,25,0.0,0.0,0.0,0.06,0.00,...,0.0,0.0,2013,2735,561000,0.000000e+00,0.01460,8190.60,0.000000,0.243333
15603,Îles Salomon,Vin,2655,vegetal,25,0.0,0.0,0.0,0.07,0.00,...,0.0,0.0,2013,2655,561000,0.000000e+00,0.00000,0.00,0.000000,0.000000


## Question 5 : 

In [11]:
# Changer les valeurs "inf" en Nan puis en 0
main_df = main_df.replace(np.inf, 0)

df_5 = main_df.copy()
df_5 = df_5[df_5['pourcentage_proteine'] != 0]
df_5 = df_5.groupby(df_5['Produit','']).mean()
df_5 = df_5.sort_values(by=['pourcentage_proteine'], ascending=False)
df_5.head(20)

,"(Code Produit, )","(Code zone, )","(Valeur, Aliments pour animaux)","(Valeur, Autres utilisations (non alimentaire))","(Valeur, Disponibilité alimentaire (Kcal/personne/jour))","(Valeur, Disponibilité alimentaire en quantité (kg/personne/an))","(Valeur, Disponibilité de matière grasse en quantité (g/personne/jour))","(Valeur, Disponibilité de protéines en quantité (g/personne/jour))","(Valeur, Disponibilité intérieure)","(Valeur, Exportations - Quantité)",...,"(Valeur, Traitement)","(Valeur, Variation de stock)",Année,Code produit,Population_totale,disp_alim_kcal_pays,disp_alim_kg_an,disp_alim_kg_pays,ratio kcal/kg,pourcentage_proteine
"(Produit, )",,,,,,,,,,,,,,,,,,,,,
Graines Colza/Moutarde,2558.0,39.000000,246.333333,0.000000,12.000000,1.000000,1.000000,1.000000,2699.333333,2378.333333,...,2233.333333,-993.333333,2013.0,2558.0,1.191467e+07,6.917042e+10,0.365000,4.348853e+06,1521.947259,0.365000
Soja,2555.0,128.663866,134.949580,5.319328,9.117647,1.065042,0.401008,0.921933,1886.142857,788.243697,...,1582.310924,-8.268908,2013.0,2555.0,5.134948e+07,3.028863e+11,0.336505,2.895287e+07,1657.798438,0.272545
Arachides Decortiquees,2556.0,126.048485,0.006061,7.666667,22.193939,1.464061,1.844848,0.969515,179.060606,16.448485,...,81.751515,-11.915152,2013.0,2556.0,4.135659e+07,3.936898e+11,0.353873,1.701882e+07,4700.572203,0.250913
Pois,2547.0,127.292857,23.550000,0.200000,9.835714,1.052357,0.048143,0.645571,74.414286,33.314286,...,0.000000,-4.464286,2013.0,2547.0,4.613624e+07,1.387711e+11,0.235634,9.276916e+06,2730.391802,0.224239
Légumineuses Autres,2549.0,125.481013,41.303797,5.164557,31.208861,3.335759,0.210886,2.004747,242.721519,33.018987,...,0.000000,1.006329,2013.0,2549.0,4.270186e+07,5.965386e+11,0.731733,3.637571e+07,2811.427686,0.223047
Haricots,2546.0,125.459459,15.925676,0.398649,30.337838,3.276284,0.138041,1.955811,150.385135,25.797297,...,0.000000,1.810811,2013.0,2546.0,4.362494e+07,3.974010e+11,0.713871,2.538860e+07,2845.032136,0.218658
"Plantes Oleiferes, Autre",2570.0,144.343750,10.125000,3.437500,4.328125,0.417031,0.334219,0.203750,10.843750,80.046875,...,38.515625,0.484375,2013.0,2570.0,3.198395e+07,5.336228e+10,0.074369,3.179955e+06,2175.044652,0.187802
Sésame,2561.0,127.990099,0.000000,0.188119,7.287129,0.462376,0.637822,0.226238,38.217822,11.366337,...,21.722772,-2.247525,2013.0,2561.0,5.557833e+07,8.413732e+10,0.082577,2.477477e+06,3959.725827,0.187107
"Viande, Autre",2735.0,126.968553,0.352201,0.094340,6.723270,1.970629,0.293774,0.947799,42.817610,4.830189,...,0.106918,0.018868,2013.0,2735.0,4.322230e+07,5.046576e+10,0.345947,7.607044e+06,912.642832,0.178481


Parmis les 20 aliments les plus riches en protéine on retrouve : 'Graines Colza/Moutarde', 'viande de bovin', 'Arachides Decortiquees', 'Pois', 'Légumineuses Autres'

In [12]:
df_5 = df_5.sort_values(by=['ratio kcal/kg'], ascending=False)
df_5.head(20)

,"(Code Produit, )","(Code zone, )","(Valeur, Aliments pour animaux)","(Valeur, Autres utilisations (non alimentaire))","(Valeur, Disponibilité alimentaire (Kcal/personne/jour))","(Valeur, Disponibilité alimentaire en quantité (kg/personne/an))","(Valeur, Disponibilité de matière grasse en quantité (g/personne/jour))","(Valeur, Disponibilité de protéines en quantité (g/personne/jour))","(Valeur, Disponibilité intérieure)","(Valeur, Exportations - Quantité)",...,"(Valeur, Traitement)","(Valeur, Variation de stock)",Année,Code produit,Population_totale,disp_alim_kcal_pays,disp_alim_kg_an,disp_alim_kg_pays,ratio kcal/kg,pourcentage_proteine
"(Produit, )",,,,,,,,,,,,,,,,,,,,,
Huile d'Arachide,2572.0,255.000000,0.000000,2.000000,9.000000,0.360000,0.990000,0.010000,6.000000,16.000000,...,0.000000,-1.000000,2013.0,2572.0,1.110400e+07,3.647664e+10,0.003650,4.052960e+04,9119.160000,0.010139
Huile de Tournesol,2573.0,117.157895,0.000000,50.368421,198.157895,8.181053,22.381053,0.045263,304.789474,343.157895,...,0.000000,28.578947,2013.0,2573.0,2.663947e+07,2.233419e+12,0.016521,4.529264e+05,8828.241740,0.002841
Huile de Germe de Maïs,2582.0,39.000000,0.000000,20.500000,101.000000,4.265000,11.350000,0.075000,45.000000,5.000000,...,0.000000,0.000000,2013.0,2582.0,6.960500e+06,2.112313e+11,0.027375,1.597386e+05,8720.597381,0.007632
Huile de Soja,2571.0,129.400000,0.000000,186.920000,149.920000,6.333600,16.920800,0.052000,695.920000,136.320000,...,0.000000,11.040000,2013.0,2571.0,4.250568e+07,4.262362e+12,0.018980,1.188547e+06,8615.155413,0.003226
Huile de Palme,2577.0,122.379310,0.000000,232.827586,91.000000,3.831724,10.264138,0.043448,347.310345,853.068966,...,0.103448,-16.655172,2013.0,2577.0,2.558872e+07,9.638855e+11,0.015859,4.876330e+05,8606.158282,0.004992
Huile de Colza&Moutarde,2574.0,118.764706,41.176471,180.176471,132.823529,5.591765,15.008824,0.032941,404.176471,340.470588,...,0.058824,0.176471,2013.0,2574.0,3.865076e+07,1.565605e+12,0.012024,3.384563e+05,8570.661604,0.003944
Huile Graines de Coton,2575.0,150.428571,0.000000,11.142857,53.857143,2.274286,6.107143,0.012857,150.714286,4.714286,...,0.000000,1.428571,2013.0,2575.0,7.342300e+07,1.162265e+12,0.004693,4.579431e+05,8551.950037,0.005612
Huile de Palmistes,2576.0,171.400000,0.000000,9.000000,13.400000,0.590000,1.506000,0.010000,36.400000,19.200000,...,0.000000,1.800000,2013.0,2576.0,4.739940e+07,2.270225e+11,0.003650,1.730078e+05,8550.487825,0.006449
Huile de Coco,2578.0,136.538462,0.000000,12.153846,37.384615,1.553846,4.214615,0.029231,32.000000,10.615385,...,0.000000,0.000000,2013.0,2578.0,2.947600e+07,1.619391e+11,0.010669,1.408650e+05,8403.109571,0.007364


Parmis les 20 aliments les plus caloriques on retrouve : 'Arachides Decortiquees', 'Sésame', 'Feve de Cacao', 'Riz (Eq Blanchi)','Millet'

## Question 6 :

In [13]:
df_6 = main_df.copy()
df_6 = df_6[df_6['Type',''] == 'vegetal'].copy()
df_6['Valeur','Disponibilité intérieure'] = df_6['Valeur','Disponibilité intérieure'] * 1000 *1000
df_6['Disponibilité_interieur_kcal'] = df_6['Valeur','Disponibilité intérieure'] * df_6['ratio kcal/kg']
disp_world_kcal = df_6['Disponibilité_interieur_kcal'].sum()
disp_world_kcal

1.2312091265143276e+16

## Question 7 :

In [14]:
# estimation du besoin calorique d'un humain sur 1 an 
human_kcal_year = 2250 * 365

# calcul du nombre d'humains pouvant être nourris par an en termes de kcal
world_human_kcal = disp_world_kcal /human_kcal_year  
world_human_kcal

14991891951.46822

In [17]:
# poids moyen d'un être humain dans le monde
moy_weight_human = 62

# besoin moyen d'un être humain en protéine sur 1 an
human_prot_year = (0.8 / 1000) * moy_weight_human * 365 

# Calcul de la disponibilité intérieure végétale en protéine 
df_6['Disponibilité_intérieur_prot'] = df_6['Valeur','Disponibilité intérieure'] * df_6['pourcentage_proteine']

disp_world_prot = df_6['Disponibilité_intérieur_prot'].sum()

human_world_prot = disp_world_prot / human_prot_year
human_world_prot


16128801690.255905

In [18]:
# pourcentage par rapport à la population mondiale 
percentage_cal = (world_human_kcal *100) / nb_people
percentage_cal

214.2517291815219

environs 214 % de la population mondiale peut être nourrie en termes de kcal

In [19]:
percentage_prot = (human_world_prot * 100 ) / nb_people
percentage_prot

230.49950352828932

environs 230 % de la population peut être nourrie en termes de protéines 

## Question 8 :

Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux, la nourriture végétale destinée aux animaux et les pertes de produits végétaux étaient utilisés pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale

In [20]:
df_8 = df_6.copy()

# additionner les différentes valeurs 'disponibilité alimentaire', 'nourriture destinée aux animaux' et les 'Pertes' pour chaque pays
# créer une colonne pour la disponibilité alimentaire totale  en kg  par pays .

df_8['disp_alim_kg_total'] = df_8['Valeur','Disponibilité alimentaire en quantité (kg/personne/an)'] * df_8['Population_totale']

df_8['poids_alim_anim_pertes_total'] = df_8['disp_alim_kg_total'] + (df_8['Valeur','Aliments pour animaux'] * 1000000) + (df_8['Valeur','Pertes'] * 1000000)
df_8['poids_alim_anim_pertes_total_kcal'] = df_8['poids_alim_anim_pertes_total'] * df_8['ratio kcal/kg']
poids_alim_anim_pertes_total_kcal = df_8['poids_alim_anim_pertes_total_kcal'].sum()
disp_anim_loss_kcal_world = poids_alim_anim_pertes_total_kcal / human_kcal_year
disp_anim_loss_kcal_percentage = (disp_anim_loss_kcal_world * 100/ nb_people)
disp_anim_loss_kcal_percentage


159.66047343018025

Le pourcentage représente 159% de la population mondiale qui pourrait être nourrie en termes de calories

In [22]:
# résultat en termes de proteine 
df_8['poids_alim_anim_pertes_total_prot'] = df_8['poids_alim_anim_pertes_total'] * df_8['pourcentage_proteine']
poids_alim_anim_pertes_total_prot = df_8['poids_alim_anim_pertes_total_prot'].sum()
disp_anim_loss_prot_world = poids_alim_anim_pertes_total_prot / human_prot_year
percentage_prot_world = (disp_anim_loss_prot_world  * 100 / nb_people)
percentage_prot_world

161.34080674604547

Le pourcentage représente 161% de la population mondiale qui pourrait être nourrie en termes de protéines

## Question 9 :


Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [23]:
# calculez la disponibilité alimentaire mondiale kcal 

df_9 = main_df.copy()

disp_alim_kcal_totale = df_9['disp_alim_kcal_pays'].sum()
disp_alim_kcal_world = disp_alim_kcal_totale /  human_kcal_year

disp_alim_kcal_world_percentage  = (disp_alim_kcal_world * 100)/ nb_people

disp_alim_kcal_world_percentage

128.14433570131726

cela représente 128 % de la population mondiale qui pourraient être nourri en terme de calories

In [24]:
# disponibilité alimentaire  mondiale prot
disp_alim_prot_totale = df_9['disp_alim_kg_pays'].sum()
disp_alim_prot_world = disp_alim_prot_totale /  human_prot_year
disp_alim_prot_world_percentage = (disp_alim_prot_world * 100 )/ nb_people
disp_alim_prot_world_percentage 

163.60897618362563

cela  représente 163% de la population qui pourrait nourrie en termes de protéines

## Question 10 :


A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [25]:
df_10 = sous_alimentation
df_10[df_10['Valeur'] == '<0.1'] = 0
df_10['Valeur'] = pd.to_numeric(df_10['Valeur'])
df_10.fillna(0, inplace = True)


df_10['population'] = df_10['Valeur'] * 1000000
df_10 = df_10[df_10['Année'] == '2012-2014']
df_10 = df_10[df_10['Zone'] != 'Chine']
population_mondial_sous_alim  = df_10['population'].sum()
population_mondial_sous_alim_percentage = (population_mondial_sous_alim * 100)/nb_people
population_mondial_sous_alim_percentage

10.628345742359295

Les personnes étant en sous alimentation représente environs 10% de la population mondiale

# Question 11 :

En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [26]:
code_cerales = cereal['Code Produit'].unique()
df_11 = main_df[main_df['Code Produit',''].isin(code_cerales)]
poids_alim_anim = df_11['Valeur', 'Aliments pour animaux'].sum() *1000000
poids_nourriture = df_11['Valeur', 'Nourriture'].sum() *1000000
poids_totale = poids_alim_anim + poids_nourriture
alim_anim_percentage = (poids_alim_anim * 100)/poids_totale
alim_anim_percentage

45.91402568664604

45% est destiné à l'alimentation animale

# Question 12 : 


Sélectionnez parmi les données des bilans alimentaires les informations relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition

In [27]:
# pays avec des personnes en sous_nutrition 
df_12 = df_10.copy()
df_12 = df_12[df_12['population'] > 0]

# Sélection des informations relatives à ces pays
df_12_bis = main_df[main_df['Zone',''].isin(df_12['Zone']) ]
df_12_bis

,"(Zone, )","(Produit, )","(Code Produit, )","(Type, )","(Code zone, )","(Valeur, Aliments pour animaux)","(Valeur, Autres utilisations (non alimentaire))","(Valeur, Disponibilité alimentaire (Kcal/personne/jour))","(Valeur, Disponibilité alimentaire en quantité (kg/personne/an))","(Valeur, Disponibilité de matière grasse en quantité (g/personne/jour))",...,"(Valeur, Traitement)","(Valeur, Variation de stock)",Année,Code produit,Population_totale,disp_alim_kcal_pays,disp_alim_kg_an,disp_alim_kg_pays,ratio kcal/kg,pourcentage_proteine
0,Afghanistan,Abats Comestible,2736,animal,2,0.0,0.0,5.0,1.72,0.20,...,0.0,0.0,2013,2736,30552000,5.575740e+10,0.28105,8586639.60,1052.026415,0.163401
1,Afghanistan,"Agrumes, Autres",2614,vegetal,2,0.0,0.0,1.0,1.29,0.01,...,0.0,0.0,2013,2614,30552000,1.115148e+10,0.00730,223029.60,285.935385,0.005659
2,Afghanistan,Aliments pour enfants,2680,vegetal,2,0.0,0.0,1.0,0.06,0.01,...,0.0,0.0,2013,2680,30552000,1.115148e+10,0.01095,334544.40,5575.740000,0.182500
3,Afghanistan,Ananas,2618,vegetal,2,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,2013,2618,30552000,0.000000e+00,0.00000,0.00,0.000000,0.000000
4,Afghanistan,Bananes,2615,vegetal,2,0.0,0.0,4.0,2.70,0.02,...,0.0,0.0,2013,2615,30552000,4.460592e+10,0.01825,557574.00,543.974634,0.006759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15522,Éthiopie,Viande de Suides,2733,animal,238,0.0,0.0,0.0,0.02,0.02,...,0.0,0.0,2013,2733,94101000,0.000000e+00,0.00365,343468.65,0.000000,0.182500
15523,Éthiopie,Viande de Volailles,2734,animal,238,0.0,0.0,2.0,0.66,0.14,...,0.0,0.0,2013,2734,94101000,6.869373e+10,0.08030,7556310.30,1107.963387,0.121667
15524,Éthiopie,"Viande, Autre",2735,animal,238,0.0,0.0,4.0,1.20,0.18,...,0.0,0.0,2013,2735,94101000,1.373875e+11,0.20075,18890775.75,1215.818230,0.167292
15525,Éthiopie,Vin,2655,vegetal,238,0.0,0.0,0.0,0.02,0.00,...,0.0,0.0,2013,2655,94101000,0.000000e+00,0.00000,0.00,0.000000,0.000000


Repérez les 15 produits les plus exportés par ce groupe de pays.

In [28]:
df_12_bis['Exportation'] = df_12_bis['Valeur','Exportations - Quantité']
df_12_bis['Importation'] = df_12_bis['Valeur','Importations - Quantité']

df_12_bis['Code produit']
expo = df_12_bis.groupby(by=[df_12_bis['Code produit'], df_12_bis['Produit', '']]).sum().reset_index()
exportation = expo.sort_values(by='Exportation',ascending=False)

exportation_15 = exportation.iloc[:15,:]
exportation_15.reset_index()
exportation_15


,Code produit,"(Produit, )","(Code Produit, )","(Code zone, )","(Valeur, Aliments pour animaux)","(Valeur, Autres utilisations (non alimentaire))","(Valeur, Disponibilité alimentaire (Kcal/personne/jour))","(Valeur, Disponibilité alimentaire en quantité (kg/personne/an))","(Valeur, Disponibilité de matière grasse en quantité (g/personne/jour))","(Valeur, Disponibilité de protéines en quantité (g/personne/jour))",...,"(Valeur, Variation de stock)",Année,Population_totale,disp_alim_kcal_pays,disp_alim_kg_an,disp_alim_kg_pays,ratio kcal/kg,pourcentage_proteine,Exportation,Importation
38,2577,Huile de Palme,242238,11909,0.0,24743.0,6777.0,281.48,766.38,0.29,...,-875.0,189222,5358550000,1.124488e+14,0.10585,7.982941e+06,620553.998179,0.019863,46285.0,31784.0
2,2514,Maïs,241344,12151,262845.0,46095.0,24842.0,2924.63,245.52,634.82,...,-13507.0,193248,5367394000,3.208767e+14,231.70930,8.029197e+09,279227.966883,7.214009,37888.0,61247.0
9,2532,Manioc,227880,11238,75733.0,34090.0,8523.0,3082.01,11.48,63.65,...,-656.0,181170,5172728000,8.935436e+13,23.23225,5.933880e+08,60584.227887,0.440939,35843.0,34484.0
95,2805,Riz (Eq Blanchi),269280,12151,32927.0,17838.0,38160.0,3907.91,92.10,747.59,...,-16734.0,193248,5367394000,1.276978e+15,272.87035,2.395291e+10,346804.428330,6.716922,32787.0,28132.0
16,2542,Sucre Eq Brut,244032,12151,23.0,6094.0,20218.0,2072.36,0.00,0.00,...,-4304.0,193248,5367394000,3.167883e+14,0.00000,0.000000e+00,343938.811803,0.000000,25237.0,38806.0
0,2511,Blé,241056,12151,48158.0,10113.0,44378.0,5626.10,215.40,1292.47,...,-3841.0,193248,5367394000,9.343356e+14,471.75155,2.785908e+10,274697.866800,7.872466,24841.0,103304.0
47,2605,"Légumes, Autres",250080,12151,37584.0,59.0,3553.0,5091.78,28.71,175.49,...,155.0,193248,5367394000,1.563192e+14,64.05385,8.622313e+09,25647.551618,1.280835,19948.0,6114.0
52,2615,Bananes,237965,11438,828.0,116.0,2154.0,1313.74,7.55,29.64,...,480.0,183183,5104293000,4.184854e+13,10.81860,5.588058e+08,49373.328631,0.732708,17805.0,3329.0
58,2625,"Fruits, Autres",252000,12151,43.0,62.0,2442.0,2084.29,40.60,29.82,...,401.0,193248,5367394000,6.041909e+13,10.88430,6.989665e+08,41524.127157,0.492380,14590.0,8416.0
96,2848,Lait - Excl Beurre,273408,12151,40082.0,1556.0,11010.0,6646.93,599.44,617.18,...,278.0,193248,5367394000,1.838078e+14,225.27070,1.082195e+10,58472.347778,3.234946,14486.0,42552.0


Parmis les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné

In [29]:

importation = df_12_bis[df_12_bis['Produit',''].isin(exportation_15['Produit',''])]

importation = importation.sort_values(by='Importation',ascending=False)
importation_200 = importation.iloc[:200,:]
importation_200

,"(Zone, )","(Produit, )","(Code Produit, )","(Type, )","(Code zone, )","(Valeur, Aliments pour animaux)","(Valeur, Autres utilisations (non alimentaire))","(Valeur, Disponibilité alimentaire (Kcal/personne/jour))","(Valeur, Disponibilité alimentaire en quantité (kg/personne/an))","(Valeur, Disponibilité de matière grasse en quantité (g/personne/jour))",...,Année,Code produit,Population_totale,disp_alim_kcal_pays,disp_alim_kg_an,disp_alim_kg_pays,ratio kcal/kg,pourcentage_proteine,Exportation,Importation
3353,"Chine, continentale",Soja,2555,vegetal,41,9530.0,0.0,35.0,3.66,1.18,...,2013,2555,1385567000,1.770062e+13,1.20085,1.663858e+09,3489.869563,0.328101,250.0,63381.0
3323,"Chine, continentale",Manioc,2532,vegetal,41,22868.0,7940.0,6.0,1.91,0.01,...,2013,2532,1385567000,3.034392e+12,0.01825,2.528660e+07,1147.218045,0.009555,41.0,29046.0
15071,Égypte,Blé,2511,vegetal,59,4860.0,0.0,1175.0,146.83,6.27,...,2013,2511,82056000,3.519177e+13,13.00130,1.066835e+09,2920.963396,0.088547,191.0,10331.0
6226,Inde,Huile de Palme,2577,vegetal,100,0.0,6782.0,31.0,1.30,3.54,...,2013,2577,1252140000,1.416796e+13,0.00000,0.000000e+00,8686.673268,0.000000,143.0,8568.0
3320,"Chine, continentale",Lait - Excl Beurre,2848,animal,41,1608.0,0.0,58.0,32.66,3.22,...,2013,2848,1385567000,2.933245e+13,1.12055,1.552597e+09,648.202364,0.034310,129.0,8207.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,Arabie saoudite,Tomates,2601,vegetal,194,0.0,0.0,16.0,29.42,0.18,...,2013,2601,28829000,1.683614e+11,0.30295,8.733746e+06,198.539340,0.010297,23.0,394.0
708,Argentine,Bananes,2615,vegetal,9,0.0,0.0,22.0,12.84,0.04,...,2013,2615,41446000,3.328114e+11,0.10220,4.235781e+06,625.585301,0.007960,0.0,392.0
14176,Tunisie,Sucre Eq Brut,2542,vegetal,222,0.0,0.0,330.0,33.88,0.00,...,2013,2542,10997000,1.324589e+12,0.00000,0.000000e+00,3551.176005,0.000000,8.0,385.0
679,Arabie saoudite,Pommes,2617,vegetal,194,0.0,0.0,10.0,10.46,0.04,...,2013,2617,28829000,1.052258e+11,0.01825,5.261293e+05,348.429967,0.001745,59.0,380.0


Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. Ensuite, calculez pour chaque produit les 2 quantités suivantes : le ratio entre la quantité destinée aux "Autres utilisations" (Other uses) et la disponibilité intérieure. le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

In [30]:
importation_by_product = importation_200.groupby(importation_200['Produit','']).sum()
importation_by_product

,"(Code Produit, )","(Code zone, )","(Valeur, Aliments pour animaux)","(Valeur, Autres utilisations (non alimentaire))","(Valeur, Disponibilité alimentaire (Kcal/personne/jour))","(Valeur, Disponibilité alimentaire en quantité (kg/personne/an))","(Valeur, Disponibilité de matière grasse en quantité (g/personne/jour))","(Valeur, Disponibilité de protéines en quantité (g/personne/jour))","(Valeur, Disponibilité intérieure)","(Valeur, Exportations - Quantité)",...,Année,Code produit,Population_totale,disp_alim_kcal_pays,disp_alim_kg_an,disp_alim_kg_pays,ratio kcal/kg,pourcentage_proteine,Exportation,Importation
"(Produit, )",,,,,,,,,,,,,,,,,,,,,
Bananes,10460,377,0.0,21.0,113.0,67.77,0.52,1.43,14255.0,44.0,...,8052,10460,1513806000,7.521404e+12,0.52195,9.449572e+07,2464.273119,0.033068,44.0,1927.0
Blé,125550,6728,44858.0,9892.0,29944.0,3788.25,135.75,879.82,294641.0,7769.0,...,100650,125550,3649026000,6.538242e+14,321.13430,1.971949e+10,142202.025200,4.157464,7769.0,96333.0
"Fruits, Autres",18375,1069,0.0,41.0,195.0,167.17,2.16,2.31,64138.0,5282.0,...,14091,18375,1701236000,2.427948e+13,0.84315,2.866121e+08,2971.294651,0.034891,5282.0,4464.0
Huile de Palme,38655,1911,0.0,17886.0,1135.0,47.90,128.33,0.16,26450.0,20231.0,...,30195,38655,3741856000,7.151876e+13,0.05840,7.063217e+06,102838.111763,0.009931,20231.0,26634.0
Lait - Excl Beurre,59808,3073,3600.0,1326.0,2094.0,1363.70,110.70,122.01,103920.0,5976.0,...,42273,59808,2681300000,5.732394e+13,44.53365,3.243225e+09,11902.954602,0.696855,5976.0,35737.0
"Légumes, Autres",10420,673,1.0,47.0,150.0,172.69,1.16,6.88,14565.0,863.0,...,8052,10420,355940000,4.280877e+12,2.51120,1.954233e+08,1288.698645,0.058415,863.0,2112.0
Manioc,12660,703,25152.0,21068.0,184.0,63.97,0.38,0.93,66296.0,26161.0,...,10065,12660,1755491000,1.611638e+13,0.33945,9.298549e+07,4363.234913,0.026082,26161.0,33566.0
Maïs,65364,3094,215133.0,40411.0,5944.0,692.55,63.15,151.33,334220.0,1333.0,...,52338,65364,2563515000,1.465291e+14,55.23545,3.625003e+09,78304.391586,1.979151,1333.0,56510.0
"Oranges, Mandarines",2611,194,0.0,0.0,8.0,10.87,0.03,0.13,322.0,321.0,...,2013,2611,28829000,8.418068e+10,0.04745,1.367936e+06,268.947859,0.004365,321.0,595.0


 calculer le ratio entre la quantité destinée aux "Autres utilisations" (Other uses) et la disponibilité intérieure.

In [31]:
importation_by_product['ratio_other_disp'] = importation_by_product['Valeur','Autres utilisations (non alimentaire)'] / importation_by_product['Valeur','Disponibilité intérieure']
importation_by_product

,"(Code Produit, )","(Code zone, )","(Valeur, Aliments pour animaux)","(Valeur, Autres utilisations (non alimentaire))","(Valeur, Disponibilité alimentaire (Kcal/personne/jour))","(Valeur, Disponibilité alimentaire en quantité (kg/personne/an))","(Valeur, Disponibilité de matière grasse en quantité (g/personne/jour))","(Valeur, Disponibilité de protéines en quantité (g/personne/jour))","(Valeur, Disponibilité intérieure)","(Valeur, Exportations - Quantité)",...,Code produit,Population_totale,disp_alim_kcal_pays,disp_alim_kg_an,disp_alim_kg_pays,ratio kcal/kg,pourcentage_proteine,Exportation,Importation,ratio_other_disp
"(Produit, )",,,,,,,,,,,,,,,,,,,,,
Bananes,10460,377,0.0,21.0,113.0,67.77,0.52,1.43,14255.0,44.0,...,10460,1513806000,7.521404e+12,0.52195,9.449572e+07,2464.273119,0.033068,44.0,1927.0,0.001473
Blé,125550,6728,44858.0,9892.0,29944.0,3788.25,135.75,879.82,294641.0,7769.0,...,125550,3649026000,6.538242e+14,321.13430,1.971949e+10,142202.025200,4.157464,7769.0,96333.0,0.033573
"Fruits, Autres",18375,1069,0.0,41.0,195.0,167.17,2.16,2.31,64138.0,5282.0,...,18375,1701236000,2.427948e+13,0.84315,2.866121e+08,2971.294651,0.034891,5282.0,4464.0,0.000639
Huile de Palme,38655,1911,0.0,17886.0,1135.0,47.90,128.33,0.16,26450.0,20231.0,...,38655,3741856000,7.151876e+13,0.05840,7.063217e+06,102838.111763,0.009931,20231.0,26634.0,0.676219
Lait - Excl Beurre,59808,3073,3600.0,1326.0,2094.0,1363.70,110.70,122.01,103920.0,5976.0,...,59808,2681300000,5.732394e+13,44.53365,3.243225e+09,11902.954602,0.696855,5976.0,35737.0,0.012760
"Légumes, Autres",10420,673,1.0,47.0,150.0,172.69,1.16,6.88,14565.0,863.0,...,10420,355940000,4.280877e+12,2.51120,1.954233e+08,1288.698645,0.058415,863.0,2112.0,0.003227
Manioc,12660,703,25152.0,21068.0,184.0,63.97,0.38,0.93,66296.0,26161.0,...,12660,1755491000,1.611638e+13,0.33945,9.298549e+07,4363.234913,0.026082,26161.0,33566.0,0.317787
Maïs,65364,3094,215133.0,40411.0,5944.0,692.55,63.15,151.33,334220.0,1333.0,...,65364,2563515000,1.465291e+14,55.23545,3.625003e+09,78304.391586,1.979151,1333.0,56510.0,0.120911
"Oranges, Mandarines",2611,194,0.0,0.0,8.0,10.87,0.03,0.13,322.0,321.0,...,2611,28829000,8.418068e+10,0.04745,1.367936e+06,268.947859,0.004365,321.0,595.0,0.000000


le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)


In [32]:
importation_by_product['ratio_anim_nourriture'] = importation_by_product['Valeur','Aliments pour animaux'] / (importation_by_product['Valeur','Aliments pour animaux'] + importation_by_product['Valeur','Nourriture'] )
importation_by_product

,"(Code Produit, )","(Code zone, )","(Valeur, Aliments pour animaux)","(Valeur, Autres utilisations (non alimentaire))","(Valeur, Disponibilité alimentaire (Kcal/personne/jour))","(Valeur, Disponibilité alimentaire en quantité (kg/personne/an))","(Valeur, Disponibilité de matière grasse en quantité (g/personne/jour))","(Valeur, Disponibilité de protéines en quantité (g/personne/jour))","(Valeur, Disponibilité intérieure)","(Valeur, Exportations - Quantité)",...,Population_totale,disp_alim_kcal_pays,disp_alim_kg_an,disp_alim_kg_pays,ratio kcal/kg,pourcentage_proteine,Exportation,Importation,ratio_other_disp,ratio_anim_nourriture
"(Produit, )",,,,,,,,,,,,,,,,,,,,,
Bananes,10460,377,0.0,21.0,113.0,67.77,0.52,1.43,14255.0,44.0,...,1513806000,7.521404e+12,0.52195,9.449572e+07,2464.273119,0.033068,44.0,1927.0,0.001473,0.000000
Blé,125550,6728,44858.0,9892.0,29944.0,3788.25,135.75,879.82,294641.0,7769.0,...,3649026000,6.538242e+14,321.13430,1.971949e+10,142202.025200,4.157464,7769.0,96333.0,0.033573,0.170348
"Fruits, Autres",18375,1069,0.0,41.0,195.0,167.17,2.16,2.31,64138.0,5282.0,...,1701236000,2.427948e+13,0.84315,2.866121e+08,2971.294651,0.034891,5282.0,4464.0,0.000639,0.000000
Huile de Palme,38655,1911,0.0,17886.0,1135.0,47.90,128.33,0.16,26450.0,20231.0,...,3741856000,7.151876e+13,0.05840,7.063217e+06,102838.111763,0.009931,20231.0,26634.0,0.676219,0.000000
Lait - Excl Beurre,59808,3073,3600.0,1326.0,2094.0,1363.70,110.70,122.01,103920.0,5976.0,...,2681300000,5.732394e+13,44.53365,3.243225e+09,11902.954602,0.696855,5976.0,35737.0,0.012760,0.036040
"Légumes, Autres",10420,673,1.0,47.0,150.0,172.69,1.16,6.88,14565.0,863.0,...,355940000,4.280877e+12,2.51120,1.954233e+08,1288.698645,0.058415,863.0,2112.0,0.003227,0.000075
Manioc,12660,703,25152.0,21068.0,184.0,63.97,0.38,0.93,66296.0,26161.0,...,1755491000,1.611638e+13,0.33945,9.298549e+07,4363.234913,0.026082,26161.0,33566.0,0.317787,0.621574
Maïs,65364,3094,215133.0,40411.0,5944.0,692.55,63.15,151.33,334220.0,1333.0,...,2563515000,1.465291e+14,55.23545,3.625003e+09,78304.391586,1.979151,1333.0,56510.0,0.120911,0.813850
"Oranges, Mandarines",2611,194,0.0,0.0,8.0,10.87,0.03,0.13,322.0,321.0,...,28829000,8.418068e+10,0.04745,1.367936e+06,268.947859,0.004365,321.0,595.0,0.000000,0.000000


Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)

In [33]:
top_ratio_anim = importation_by_product.sort_values(by='ratio_anim_nourriture', ascending=False)
top_ratio_anim.head(3)

,"(Code Produit, )","(Code zone, )","(Valeur, Aliments pour animaux)","(Valeur, Autres utilisations (non alimentaire))","(Valeur, Disponibilité alimentaire (Kcal/personne/jour))","(Valeur, Disponibilité alimentaire en quantité (kg/personne/an))","(Valeur, Disponibilité de matière grasse en quantité (g/personne/jour))","(Valeur, Disponibilité de protéines en quantité (g/personne/jour))","(Valeur, Disponibilité intérieure)","(Valeur, Exportations - Quantité)",...,Population_totale,disp_alim_kcal_pays,disp_alim_kg_an,disp_alim_kg_pays,ratio kcal/kg,pourcentage_proteine,Exportation,Importation,ratio_other_disp,ratio_anim_nourriture
"(Produit, )",,,,,,,,,,,,,,,,,,,,,
Maïs,65364,3094,215133.0,40411.0,5944.0,692.55,63.15,151.33,334220.0,1333.0,...,2563515000,1.465291e+14,55.23545,3.625003e+09,78304.391586,1.979151,1333.0,56510.0,0.120911,0.813850
Poissons Pelagiques,16578,812,10836.0,40.0,126.0,50.76,5.34,18.15,16419.0,3072.0,...,1925294000,5.065151e+12,6.62475,7.328422e+08,5628.836072,0.757834,3072.0,11692.0,0.002436,0.661579
Soja,25550,1553,10668.0,524.0,251.0,32.24,11.44,25.24,89821.0,308.0,...,2091385000,2.237415e+13,9.21260,2.102859e+09,23561.310065,2.835110,308.0,76334.0,0.005834,0.621642


les 3 aliments pour le ratio nourriture destiné aux animaux et la quantité destinée à la nourriture  sont : Maïs , Poissons Pelagique et Soja

In [34]:
top_ratio_other = importation_by_product.sort_values(by='ratio_other_disp', ascending=False)
top_ratio_other.head(3)

,"(Code Produit, )","(Code zone, )","(Valeur, Aliments pour animaux)","(Valeur, Autres utilisations (non alimentaire))","(Valeur, Disponibilité alimentaire (Kcal/personne/jour))","(Valeur, Disponibilité alimentaire en quantité (kg/personne/an))","(Valeur, Disponibilité de matière grasse en quantité (g/personne/jour))","(Valeur, Disponibilité de protéines en quantité (g/personne/jour))","(Valeur, Disponibilité intérieure)","(Valeur, Exportations - Quantité)",...,Population_totale,disp_alim_kcal_pays,disp_alim_kg_an,disp_alim_kg_pays,ratio kcal/kg,pourcentage_proteine,Exportation,Importation,ratio_other_disp,ratio_anim_nourriture
"(Produit, )",,,,,,,,,,,,,,,,,,,,,
Huile de Palme,38655,1911,0.0,17886.0,1135.0,47.90,128.33,0.16,26450.0,20231.0,...,3741856000,7.151876e+13,0.05840,7.063217e+06,102838.111763,0.009931,20231.0,26634.0,0.676219,0.000000
Manioc,12660,703,25152.0,21068.0,184.0,63.97,0.38,0.93,66296.0,26161.0,...,1755491000,1.611638e+13,0.33945,9.298549e+07,4363.234913,0.026082,26161.0,33566.0,0.317787,0.621574
Maïs,65364,3094,215133.0,40411.0,5944.0,692.55,63.15,151.33,334220.0,1333.0,...,2563515000,1.465291e+14,55.23545,3.625003e+09,78304.391586,1.979151,1333.0,56510.0,0.120911,0.813850


les 3 aliments pour le ratio entre la quantité destinée aux "Autres utilisations" (Other uses) et la disponibilité intérieure sont : Huile de palme , Manioc et Maïs

# Question 13 :


Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [35]:
test = df_11[df_11['Zone',''] == "États-Unis d'Amérique"]
poid_total_anim_usa = test['Valeur', 'Aliments pour animaux'].sum() 
(poid_total_anim_usa * 0.1) * 1000

14009600.0

cela pourrait libérer 14009600 tonnes de céreales

# Question 14 :

En Thaïlande, quelle proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?

In [36]:
thailande_info = df_12_bis[df_12_bis['Zone',''] == 'Thaïlande']
thailande_info_manioc = df_12_bis[(df_12_bis['Zone',''] == 'Thaïlande') &  (df_12_bis['Produit',''] == 'Manioc')]
thailande_info_disp_manioc = thailande_info_manioc['Valeur','Production']
thailande_info_disp_manioc

13809    30228.0
Name: (Valeur, Production), dtype: float64

In [37]:
export_manioc = (thailande_info_manioc['Exportation'] * 100) / thailande_info_disp_manioc 
export_manioc

13809    83.41273
dtype: float64

environ 83% de sa production de manioc

In [38]:
sous_nutrition_thai = df_10[df_10['Zone'] == 'Thaïlande']
sous_nutrition_thai_pop = sous_nutrition_thai['population'] 
thai = sous_nutrition_thai.iloc[0]
thai.population

5600000.0

In [39]:
thai_pop = population[population['Zone'] == 'Thaïlande']
thai_pop_value = thai_pop['Valeur'] * 1000
thai_pop_value

160    67011000
Name: Valeur, dtype: int64

In [40]:
sous_nutrition_thai_percentage = (thai.population * 100) / thai_pop_value
sous_nutrition_thai_percentage

160    8.356837
Name: Valeur, dtype: float64

cela représente environ 8% de la population thaïlandaise